In [1]:
# Presenting next friday! get 'er done!

# Ding Yanna
# dingy6@rpi.edu

import ray
import numpy as np

# https://arxiv.org/pdf/1811.00141.pdf
# ref 30 Net10 in SI table 1


ray.init(num_cpus=7, num_gpus=1)

# Hyperparams
N = 10

# I think d is the death rate. vary from 0.05 to 4
# field measurements are typically 0.1 - 0.3
d = 0

# self-limitation parameter. fixed at 1 in simulations
# changes the density by 1 / s i.e to get real answer divide by whatever s is
s = 1
alpha = 1

# Define a random adjacency matrix
# important, Aij != Aji
A = np.random.randint(0, 2, size=(N, N))

death_rate_min = 0
death_rate_max = 4
death_rate_step = 0.1

# Can't use normal here b/c negative. paper says 'a prob distribution' but not which one
# mean value is gamma (from data) and width is delta (not sure yet what that is)
# delta < gamma
# I think use a uniform dist
# okay I finanlly got it, uniform dist with mean gamma and 'width' (std)? delta
# eqn 23 in SI section 3 details the prob dist
# if delta is 0 then all weights are gamma so, 
gamma_min = 0
gamma_max = 0
gamma_step = 0
# gamma = np.random.uniform(0, 1, size=(N, N))

delta_min = 0
delta_max = 0
delta_step = 0

# integrate eqn 1 via 4th order runge-kutta algo until system reaches fixed point use t = 0.01, steady state at around 1e5
# init weights uniformly (0, 1] i guess
# similar to the process in SI section III with P(gamma_i_j)


def get_kcores(adj_matrix):
    cores = {}
    _adj_matrix = adj_matrix.copy()
    core = 1
    while _adj_matrix.sum() > 0:
        cores[core] = {'rows': [], 'cols': []}

        removed = 1
        while removed > 0:
            removed = 0
            for row in range(_adj_matrix.shape()[0]):
                if _adj_matrix[row].sum() <= core:
                    _adj_matrix[row] = 0
                    cores[core]['rows'].append(row)
                    removed += 1

            for col in range(_adj_matrix.shape()[1]):
                if _adj_matrix[:, col].sum() <= core:
                    _adj_matrix[:, col] = 0
                    cores[core]['cols'].append(col)
                    removed += 1
    return cores
    


Numerical Analysis

In [2]:
import ray

from RungeKutta import RK4

# Paper claims 1e5 iterations for stability
def solver(A, G, X, alpha, s, death_rate):
    solver = RK4(X)
    return solver.solve(y, 0.01, int(1e5))

@ray.remote
def ray_solver(A, G, X, alpha, s, death_rate):
    return solver(A, G, alpha, s, death_rate)

def K_gamma(gamma, d):
    return alpha * s (gamma + d) / (gamma - d) ** 2

def f(i, t, x, y):
    outer_sum = 0
    for j1 in range(N):

        inner_sum = 0
        for j2 in range(N):
            inner_sum += A[i][j2] * x[j2]

        outer_sum += A[i][j1] * gamma[i][j1] * x[j1] / (alpha + inner_sum)
    return - d/s + 1/s + outer_sum

# Create system of equations
xs = []
for i in range(N):
    xs.append(lambda t, x, y: f(i, t, x, y))

jobs = []
for death_rate in np.arange(death_rate_min, death_rate_max, death_rate_step):
    for gamma in np.arange(gamma_min, gamma_max, gamma_step):
        for delta in np.arange(delta_min, delta_max, delta_step):
            G = np.random.uniform(gamma - delta, gamma + delta, size=(N, N))
            jobs.append(ray_solver.remote(A, G, alpha, s, death_rate))

ray.get(jobs)

Stability Analysis

Depends on fixed point eqns

TODO
review stability theory [1]